# Kaggle Neuroblastoma Detection
## Lachlan Dryburgh 2021

Tensorflow implementation of a u-net image segmentation convolutional nerual network.  Trained to label neurons, astrocytes and neuroglioblastoma cell in microscope images.

## Imports and Defines

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os


IMG_HEIGHT = 540
IMG_WIDTH = 704
NUM_CLASS = 3

## Define out model
We are actually defining 2 models.

The downstack of the u-net is used as image classifier so that it can be trained where we only have labels for the enire image rather than pixels.  This will allow transfer learning.

The Downstack feeds back into the upstack for our unet pixel classifier.


In [10]:
in1 = keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1))

conv1 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
conv1 = layers.Dropout(0.2)(conv1)
conv1 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2))(conv1)

conv2 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
conv2 = layers.Dropout(0.2)(conv2)
conv2 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
pool2 = layers.MaxPooling2D((2, 2))(conv2)

conv3 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
conv3 = layers.Dropout(0.2)(conv3)
conv3 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
pool3 = layers.MaxPooling2D((2, 2))(conv3)

conv4 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
conv4 = layers.Dropout(0.2)(conv4)
conv4 = layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)

up1 = layers.UpSampling2D((2, 2))(conv4)
pad = layers.ZeroPadding2D(padding=((1,0),(0,0)))(up1)
up1 = layers.concatenate([pad, conv3], axis=-1)
conv5 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
conv5 = layers.Dropout(0.2)(conv5)
conv5 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)

up2 = layers.concatenate([layers.UpSampling2D((2, 2))(conv5), conv2], axis=-1)
conv6 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
conv6 = layers.Dropout(0.2)(conv6)
conv6 = layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

up3 = layers.concatenate([layers.UpSampling2D((2, 2))(conv6), conv1], axis=-1)
conv7 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up3)
conv7 = layers.Dropout(0.2)(conv7)
conv7 = layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
segmentation = layers.Conv2D(3, (1, 1), activation='sigmoid', name='seg')(conv7)

class_box = layers.Flatten()(conv4)
class_box = layers.Dense(128, activation = 'relu')(class_box)
class_box = layers.Dense(NUM_CLASS, activation = 'softmax')(class_box)

class_model = keras.Model(inputs=[in1], outputs=[class_box])

model = keras.Model(inputs=[in1], outputs=[segmentation])



# Compile The class box model


In [11]:
class_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [12]:
keras.utils.plot_model(class_model, "mini_resnet.png", show_shapes=True)

In [13]:
losses = {'seg': 'sparse_categorical_crossentropy'
            }

metrics = {'seg': ['acc']
                }
model.compile(optimizer="adam", loss = losses, metrics=metrics)

In [14]:
keras.utils.plot_model(model, "mini_resnet.png", show_shapes=True)